In [99]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,roc_auc_score

In [100]:
entprise_info=pd.read_csv("entprise_info.csv")
entprise_evaluate=pd.read_csv("entprise_evaluate.csv")
base_info=pd.read_csv("base_info.csv")

In [101]:
for column in base_info.columns:
    print(column,base_info[column].nunique(dropna=False))

id 24865
oplocdistrict 16
industryphy 20
industryco 346
dom 23278
opscope 20815
enttype 17
enttypeitem 32
opfrom 6620
opto 5747
state 6
orgid 78
jobid 434
adbusign 2
townsign 2
regtype 3
empnum 63
compform 3
parnum 52
exenum 51
opform 34
ptbusscope 1
venind 4
enttypeminu 27
midpreindcode 1
protype 3
oploc 5351
regcap 1144
reccap 598
forreccap 12
forregcap 39
congro 34
enttypegb 53


In [102]:
del base_info['ptbusscope']
del base_info['dom']
del base_info['midpreindcode']
del base_info['opscope']

In [103]:
base_info['opfrom']=pd.to_datetime(base_info.opfrom)
base_info['opfrom_year']=base_info['opfrom'].dt.year.astype('int')

In [104]:
base_info['opto']=pd.to_datetime(base_info.opto)
base_info['opto_year']=base_info['opto'].dt.year.fillna(-1).astype('int')
del base_info['opfrom']
del base_info['opto']

In [105]:
base_info['venind'].describe()

count    8437.000000
mean        2.946545
std         0.284933
min         1.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         3.000000
Name: venind, dtype: float64

In [106]:
for column in base_info.columns:
    print(column,base_info[column].isnull().sum())

id 0
oplocdistrict 0
industryphy 0
industryco 1
enttype 0
enttypeitem 8214
state 0
orgid 0
jobid 0
adbusign 0
townsign 0
regtype 0
empnum 5250
compform 14234
parnum 22526
exenum 23487
opform 15865
venind 16428
enttypeminu 17595
protype 24831
oploc 0
regcap 191
reccap 17781
forreccap 24638
forregcap 24615
congro 24616
enttypegb 0
opfrom_year 0
opto_year 0


3.0       5284
2.0       4678
1.0       2308
5.0       2059
4.0       1213
          ... 
1500.0       1
300.0        1
110.0        1
49.0         1
55.0         1
Name: empnum, Length: 62, dtype: int64

In [107]:
#空缺值处理 由于是离散型取中值无意义 用-1填充
for column in ['venind','industryco','enttypeitem','compform','enttypeminu','protype','industryco']:
    base_info[column]=base_info[column].fillna(-1).astype('int')

In [108]:
base_info.select_dtypes(['float64']).describe()

,empnum,parnum,exenum,regcap,reccap,forreccap,forregcap,congro
count,19615.000000,2339.000000,1378.000000,2.467400e+04,7.084000e+03,227.000000,250.000000,249.000000
mean,4.397961,4.587431,77.749637,5.151437e+03,4.198166e+03,292.644467,1212.583484,2805.259428
std,15.389206,8.752047,2693.742692,6.777086e+04,3.653798e+04,1841.310231,8113.267914,18131.954146
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,1.500000e+01,0.000000e+00,0.000000,0.000000,0.000000
50%,3.000000,2.000000,1.000000,8.000000e+01,0.000000e+00,0.000000,0.000000,0.000000
75%,5.000000,3.000000,3.000000,5.000000e+02,1.000000e+02,0.000000,0.000000,0.000000
max,1500.000000,100.000000,100000.000000,5.000100e+06,1.278900e+06,15428.170000,88817.920450,221453.761350


In [109]:
for column in base_info.select_dtypes(['float64']).columns:
    base_info[column]=base_info[column].fillna(base_info[column].median())

In [110]:
for column in base_info.columns:
    print(column,base_info[column].isnull().sum())

id 0
oplocdistrict 0
industryphy 0
industryco 0
enttype 0
enttypeitem 0
state 0
orgid 0
jobid 0
adbusign 0
townsign 0
regtype 0
empnum 0
compform 0
parnum 0
exenum 0
opform 15865
venind 0
enttypeminu 0
protype 0
oploc 0
regcap 0
reccap 0
forreccap 0
forregcap 0
congro 0
enttypegb 0
opfrom_year 0
opto_year 0


In [111]:
base_info.select_dtypes(['object'])

,id,industryphy,opform,oploc
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,M,NaN,2367b4cac96d8598
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,O,10,31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3...
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,R,NaN,2367b4cac96d8598
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,L,01-以个人财产出资,2367b4cac96d8598
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,R,NaN,2367b4cac96d8598
...,...,...,...,...
24860,f1c1045b13d18329a2bd99d2a7e2227688c0d69bf1d1e325,O,10,f67c1b92f52ac52e424308ab51241cdef9be3e39c8e1c6...
24861,f000950527a6feb6bde38216d7cbbf32e66d3a3a96d4dbda,J,NaN,2367b4cac96d8598
24862,da8691b210adb3f65b43370d3a362f4aa1d3b16b5ba0c9d7,O,10,2367b4cac96d8598
24863,516ab81418ed215dcbbf0614a7b929e691f8eed153d7bb31,O,NaN,2367b4cac96d8598


In [112]:
base_info['opform'] = base_info['opform'].replace('01', '01-以个人财产出资').replace('02', '02-以家庭共有财产作为个人出资')

In [113]:
vc=base_info['oploc'].value_counts(dropna=True,normalize=True)
vc.head()

2367b4cac96d8598                                                                    0.755761
31487d8f256f16bd6244b7251be2ebb27b17bdfd95c8f3b19ffcd2880f506783bee75565895bdbb5    0.013553
31487d8f256f16bd6244b7251be2ebb2ae36cd652943e86c8f0d6a96cbb90e3b                    0.001046
31487d8f256f16bd6244b7251be2ebb24d1db51663c654f53b92066f5b24275c                    0.000684
ca213febe80e171cf62c2ed671ebb5cbf19e3fe4b5809fbb                                    0.000684
Name: oploc, dtype: float64

In [114]:
for column in ['industryphy', 'opform', 'oploc', 'orgid', 'jobid', 'oplocdistrict','enttypegb', 'industryco', 'enttype', 'enttypeitem']:
    lb=LabelEncoder()
    base_info[column]=lb.fit_transform(base_info[column].astype(str))
    fre=base_info[column].value_counts(dropna=True,normalize=True).to_dict()
    base_info[f'{column}_freq']=base_info[column].map(fre)

In [115]:
del base_info['venind']
del base_info['congro']
del base_info['protype']
del base_info['parnum']
del base_info['opform']
del base_info['opform_freq']
del base_info['exenum']
del base_info['forregcap']
del base_info['forreccap']

In [116]:
entprise_evaluate.columns=['id','label'] 

In [117]:
for column in entprise_evaluate.columns:
    print(column,entprise_evaluate[column].isnull().sum())

id 0
label 10000


In [118]:
labels=pd.concat([entprise_info,entprise_evaluate])
labels.head()

,id,label
0,59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7,1.0
1,da8691b210adb3f6be8064e006f220070565db287275ad38,0.0
2,82750f1b9d122350918121f97c99bf96e11aa24ee91504a9,0.0
3,f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec,0.0
4,f1c1045b13d1832927e3743e49d2917f2d98424f0849a373,0.0


In [119]:
df=pd.merge(base_info,labels,on='id',how='left')
df[[col for col in df.columns if col!='label']].fillna(-1,inplace=True)

In [120]:
train=df[df.label.notna()]
test=df[df.label.isna()]
print(train.shape,test.shape)

(14865, 31) (10000, 31)


In [121]:
import lightgbm as lgb

In [122]:
model=lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           tree_learner='serial',
                           num_leaves=64,
                           max_depth=8,
                           learning_rate=0.02,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.8,
                           reg_alpha=0.3,
                           reg_lambda=0.5,
                           random_state=2020,
                           is_unbalance=True)

In [123]:
ycol='label'
feature_names=list(filter(lambda x: x not in [ycol,'id'],train.columns))
def f1_score_custom(y_t,y_p):
    y_p=y_p.round()
    return 'f1',f1_score(y_t,y_p),True


In [124]:
oof = []
prediction = test[['id']]
prediction[f'{ycol}_0'] = 0
prediction[f'{ycol}_1'] = 0
df_importance_list = []

In [125]:

kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=2020)
for fold_id,(trn_idx,val_idx) in enumerate(kfold.split(train[feature_names],train[ycol])):
    X_train = train.iloc[trn_idx][feature_names]
    Y_train = train.iloc[trn_idx][ycol]

    X_val = train.iloc[val_idx][feature_names]
    Y_val = train.iloc[val_idx][ycol]

    lgb_model=model.fit(X_train,Y_train,
                        eval_names=['train', 'valid'],
                        eval_set=[(X_train, Y_train), (X_val, Y_val)],
                        verbose=500,
                        eval_metric=lambda y_true, y_pred: f1_score_custom(y_true, y_pred),
                        early_stopping_rounds=50)

    pred_val=lgb_model.predict_proba(X_val,num_iteration=lgb_model.best_iteration_)
    df_oof=train.iloc[val_idx][['id',ycol]].copy()
    df_oof[f'{ycol}_0']=pred_val[:,0]
    df_oof[f'{ycol}_1']=pred_val[:,1]
    oof.append(df_oof)

    pred_test=lgb_model.predict_proba(test[feature_names],num_iteration=lgb_model.best_iteration_)
    prediction[f'{ycol}_0']+=pred_test[:,0]/kfold.n_splits
    prediction[f'{ycol}_1']+=pred_test[:,1]/kfold.n_splits

    df_importance=pd.DataFrame({
        'column':feature_names,
        'importance':lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)



[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[42]	train's binary_logloss: 0.09355	train's f1: 0.813034	valid's binary_logloss: 0.0917691	valid's f1: 0.835616
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[34]	train's binary_logloss: 0.100728	train's f1: 0.819914	valid's binary_logloss: 0.105505	valid's f1: 0.797136
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[35]	train's binary_logloss: 0.0990984	train's f1: 0.826698	valid's binary_logloss: 0.104939	valid's f1: 0.794521


In [126]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,industryco,101.4
1,regcap,75.8
2,enttypegb_freq,60.6
3,opfrom_year,51.8
4,jobid_freq,45.6
5,reccap,44.0
6,enttypegb,41.6
7,industryphy,39.8
8,industryco_freq,33.6
9,jobid,31.8


In [127]:
df_oof=pd.concat(oof)
score=f1_score(df_oof[ycol].astype('int'),np.argmax(df_oof[['label_0', 'label_1']].values, axis=1).astype('int'))


In [128]:
score

0.8084507042253521

In [129]:
res=prediction[['id','label_1']]
res.columns=['id','score']

res=pd.merge(entprise_evaluate,res,on='id',how='left')
res.drop(['label'],axis=1,inplace=True)

res.head()

,id,score
0,82750f1b9d1223508ee329d47e27d35176c93eb9f35e9c1a,0.051506
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,0.522571
2,e9f7b28ec10e04700ef4db75a494f9a1e8e8b09555e6afa1,0.031967
3,beb4aaaa89e0a0ae9d77bd5d7665be6342f552f51840cf19,0.031966
4,e9f7b28ec10e0470ee4172cec0133b6826c34f27d3dff204,0.048287


In [130]:
res.to_csv(f'baseMining2_{score}.csv',index=False)